# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846880190857                   -0.70    4.8         
  2   -7.852332210639       -2.26       -1.53    1.0   23.8ms
  3   -7.852615323113       -3.55       -2.56    1.2   23.7ms
  4   -7.852645994914       -4.51       -2.90    2.5   31.8ms
  5   -7.852646508763       -6.29       -3.19    1.2   24.4ms
  6   -7.852646679972       -6.77       -4.19    1.0   23.2ms
  7   -7.852646686681       -8.17       -4.98    2.2   31.6ms
  8   -7.852646686726      -10.35       -5.51    1.5   25.9ms
  9   -7.852646686728      -11.62       -5.63    1.2   25.2ms
 10   -7.852646686730      -11.79       -6.77    1.0   23.3ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846966699294                   -0.71           4.8         
  2   -7.852555868404       -2.25       -1.64   0.80    2.0    252ms
  3   -7.852639757291       -4.08       -2.74   0.80    1.0   21.5ms
  4   -7.852646464313       -5.17       -3.38   0.80    1.8   25.8ms
  5   -7.852646680339       -6.67       -4.26   0.80    2.0   26.6ms
  6   -7.852646686607       -8.20       -4.81   0.80    2.0   27.0ms
  7   -7.852646686724       -9.93       -5.73   0.80    1.2   23.5ms
  8   -7.852646686730      -11.24       -6.58   0.80    1.8   26.5ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.390188e+01     3.324201e+00
 * time: 0.0574190616607666
     1     1.148487e+00     2.016294e+00
 * time: 0.2752950191497803
     2    -1.464758e+00     2.430612e+00
 * time: 0.2980780601501465
     3    -3.920534e+00     1.829291e+00
 * time: 0.330625057220459
     4    -5.233562e+00     1.591761e+00
 * time: 0.3632080554962158
     5    -6.815873e+00     1.216918e+00
 * time: 0.3963179588317871
     6    -6.910347e+00     1.588131e+00
 * time: 0.41937804222106934
     7    -7.389688e+00     1.362844e+00
 * time: 0.44258785247802734
     8    -7.644655e+00     8.061368e-01
 * time: 0.47531986236572266
     9    -7.679573e+00     9.734749e-01
 * time: 0.4984149932861328
    10    -7.804687e+00     1.659150e-01
 * time: 0.5212428569793701
    11    -7.828157e+00     7.681791e-02
 * time: 0.5442979335784912
    12    -7.842069e+00     5.107665e-02
 * time: 0.5671448707580566
    13    -7.845162e+00     7.032238e-02
 * time: 0.59057998

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846843416699                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645880746                   -1.64         
  2   -7.852646686730       -6.09       -3.71    1.92s
  3   -7.852646686730      -13.26       -7.22    163ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.596660593007698e-7
|ρ_newton - ρ_scfv| = 1.7305634035576138e-7
|ρ_newton - ρ_dm|   = 9.42695013925174e-10
